In [1]:
# !pip install diffusers
# !pip install -U -q git+https://github.com/huggingface/diffusers
# !pip install transformers -U -q
# !pip install -U tokenizers==0.19

In [3]:
import sys, os
sys.path.insert(0, os.path.abspath('/flux-controlnet-inpaint/src'))
import torch
from diffusers.utils import load_image
from diffusers.pipelines.flux.pipeline_flux_controlnet_inpaint import FluxControlNetInpaintPipeline
from diffusers.models.controlnet_flux import FluxControlNetModel
from diffusers import FluxMultiControlNetModel
from diffusers.utils import load_image, check_min_version
from PIL import Image
# import cv2
import numpy as np
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
from controlnet_aux import HEDdetector, MidasDetector, MLSDdetector, OpenposeDetector, PidiNetDetector, NormalBaeDetector, LineartDetector, LineartAnimeDetector, CannyDetector, ContentShuffleDetector, ZoeDetector, MediapipeFaceDetector, SamDetector, LeresDetector, DWposeDetector
import os
from pathlib import Path
# from inpaint import get_mask

import torch
from diffusers.utils import load_image
from diffusers.pipelines.flux.pipeline_flux_controlnet_inpaint import FluxControlNetInpaintPipeline
from diffusers.models.controlnet_flux import FluxControlNetModel
from controlnet_aux import CannyDetector
from transformers import pipeline 

In [4]:
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread


def showImagesHorizontally(list_of_files, output_path='output.png'):
    fig = figure()
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        a=fig.add_subplot(1,number_of_files,i+1)
        image = (list_of_files[i])
        imshow(image,cmap='Greys_r')
        axis('off')
    plt.tight_layout()  # Adjust the layout to prevent overlapping
    plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Save the figure
    plt.close(fig)  # Close the figure to free up memory

In [5]:

base_model = 'black-forest-labs/FLUX.1-dev'
controlnet_model2 = 'InstantX/FLUX.1-dev-Controlnet-Union'

controlnet_pose = FluxControlNetModel.from_pretrained(controlnet_model2, torch_dtype=torch.float16)
controlnet = FluxMultiControlNetModel([controlnet_pose,controlnet_pose])

pipe = FluxControlNetInpaintPipeline.from_pretrained(base_model, controlnet=controlnet, torch_dtype=torch.float16)
pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power
# pipe.to("cuda")

pipe.text_encoder.to(torch.float16)
pipe.controlnet.to(torch.float16)
pipe.enable_sequential_cpu_offload()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
image = load_image('/p12.jpg').resize((1024, 1024))
openpose = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')
pose_image = openpose(image)

print("running segmentation")
segmenter = pipeline(model="mattmdjaga/segformer_b2_clothes")
segments = segmenter(image)
segment_include = ["Upper-clothes", "Skirt", "Pants", "Dress", "Belt", "Bag", "Scarf", "Right-shoe","Left-shoe","Bag"]
mask_list = [np.array(s['mask']) for s in segments if s['label'] not in segment_include]
final_mask = np.array(mask_list[0])

for mask in mask_list:
    current_mask = np.array(mask)
    final_mask = final_mask + current_mask  # Add the current mask to the final mask
    
final_array = final_mask.copy() 
final_mask = Image.fromarray(final_mask)
mask=final_mask.resize((1024, 1024))

label=str(np.random.randint(100000000))

print("running canny")
canny = CannyDetector()
canny_image = canny(image)

image=image.resize((1024, 1024))
mask=mask.resize((1024, 1024))


showImagesHorizontally([image,mask,canny_image,pose_image],'input'+label+'.png')


running segmentation


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


running canny


In [ ]:

import random
images = []
for seed in range(10, 13):
    scale = 0.8
    prompt = "clothing model"   
    negative_prompt='ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, sleeves, tights, stockings'
         
    prompt = prompt + ", indoor" if random.uniform(0,1)<0.5 else prompt + ", outdoor"
    generator = torch.Generator(device="cpu").manual_seed(seed)
    image_res = pipe(
        prompt,
        negative_prompt,
        image=image,
        control_image=[pose_image],
        control_mode=[4],
        controlnet_conditioning_scale=0.8,
        mask_image=mask,
        strength=0.95,
        num_inference_steps=20,
        guidance_scale=5,
        generator=generator,
        joint_attention_kwargs={"scale": scale},    
    ).images[0]
    image_res.resize((1024, 1024)).save('out'+str(seed)+'.png')


  0%|          | 0/19 [00:00<?, ?it/s]